## How to install Elixir in Jupyter Lab/Notebook
This assumes that Jupyter notebook is already installed; if not, `brew install jupyterlab` should do it.
1. Install Elixir (`brew install elixir`, will also install Erlang)
2. Install `ierl` (download from `https://github.com/filmor/ierl/releases/download/v0.6.0/ierl` and `cp ~/Downloads/ierl /usr/local/bin/ && chmod +x /usr/local/bin/ierl`)
3. Install the Elixir Jupyter kernel (`ierl install elixir`)
4. Start the notebook (`jupyter notebook`)

## What is Elixir?

Elixir is a functional, meta-programming-aware language built on top of the Erlang virtual machine.

## What is Erlang?

Erlang is a **soft real-time, highly concurrent, fault-tolerant, distributed functional programming language** and environment. The environment includes the OTP library, which contains much of the functionality of erlang-in-the-large, and the BEAM virtual machine, on which the compiled Erlang code runs. The BEAM virtual machine was designed specifically for Erlang, and supports certain operations natively, such as Tail Call Optimization (TCO), process linking, and concurrency primitives.

## Why Elixir rather than Erlang?

* Arguably more pleasing and readable syntax than Erlang
* First-class, very powerful macros similar in capability to those in LISPy languages. In fact, much of the Elixir language is built using macros.
* More consistent libraries than Erlang's
* Arguably better tooling than Erlang's 
* Seamless Erlang interoperability (zero-cost ability to call any Erlang functions from Elixir and vice-versa)
* It can do everything that Erlang can do and more

## Why Elixir at all?

Well... consider a partial **list of its superpowers**, which is a superset of Erlang's.

In the following list, "BEAM" is the name of the Erlang VM. Where Erlang is mentioned, you can imagine it's Elixir/Erlang, but that's too tedious to write and read, so we just use Erlang here.

* A **first-class macro system** that can be used to write DSLs and is already used to provide many Elixir expressions.

* **Everything that runs in Erlang is a BEAM process**. The process is created and scheduled (and interrupted) by BEAM, has its own stack and heap, and is incredibly small, quick to create and fast to destroy. Because each process has its own heap, garbage collection occurs on a per-process basis, unlike (say) Java, so there's no stop-the-world GC to interfere with processing.

* BEAM itself runs on **multiple processors** (using SMP), and depending on the kind of workload can scale close to linearly as processors are added. Each BEAM OS process thread maps to multiple Erlang processes. **You can literally run a million Erlang processes off a single OS thread** (and this might have happened in some applications before BEAM got SMP capabilities).

* **Every BEAM process is completely isolated** (except for message passing) from all other processes, so if it crashes, it will not affect anything outside of itself (unless specifically configured to do so). There is no shared memory, so there are no mutexes, locks, or other synchronization primitives. This makes concurrent programming vastly less complicated. And there are no coroutines, async/await, stuff like that.

* **Supervisory framework**. A BEAM process `A` is able to _link_ itself to another BEAM process `B` such that if `A` dies, `B` gets an exit signal from `A`, which will kill `B` (and vice-versa). A process can also trap exit signals to decide whether or not it wants to die or take some recovery action. This feature is the basis of the OTP supervisor framework, which provides Erlang with the ability to create arbitrarily complex supervisory trees that can be configured to restart failed processes automatically and degrade gracefully.

* **BEAM processes only communicate with each other using message queues**. Each process automatically gets a message queue when it starts up, and any other process that knows its process ID can send a message to it. This is the basis for Elixir's `GenServer` and other process frameworks. Elixir/Erlang has a `receive` keyword that supports pattern-based selective receive, such that it can decide what kind of message it is looking for and receive only messages that match that pattern, even if there are other non-matching messages in the queue.

* **No global variables**.

* **Immutable data**.

* **Pattern matching**.

* **Tail-Call Optimization (TCO)**.

* **Hot fixing**: if a function running in a process needs to be modified, in most cases all it takes is to copy the modified module to the execution directory and reload the module using an Erlang/Elixir shell attached to the running system. BEAM will complete the current execution of that function (if it's busy executing), then swap in the code of the new function, _without interrupting the process_.

* **Key-value store**: Erlang has a built-in key-value store, `ets` (Erlang Term Storage), which is analogous to Redis but doesn't duplicate its instruction set. `ets` is the closest thing to global in-memory storage that Erlang supports, and can be set up in a variety of ways (for example, read/write for the owner process, read-only for all others).

* **Database**: Erlang has a built-in database, Mnesia, which is very useful for storing things like local configuration. It's definitely not a replacement for a full-fledged relational or document database, but has the advantage that it can store Erlang terms without conversion, so it can be very fast.

## Let's get going!

In [1]:
# Data types
123 # Integer
123.45 # Float
"hello" # String
[1, 2, 3] # List
{1, 2, 3} # Tuple
%{"first_name" => "Ed", "last_name" => "Fine"} # Map (i.e. dict)
:name # Atom
%{first_name: "Ed", last_name: "Fine"} # Map (i.e. dict) using atoms for keys
%{:first_name => "Ed", :last_name => "Fine"} # Map (i.e. dict)
<<65, 66, 67>> # Binary
'ABC' = [65, 66, 67] # Character list

'ABC'

In [2]:
# Hello, world!
IO.puts "Hello, world!"  # parens optional

Hello, world!


:ok

In [3]:
# Binding (not assignment!)
msg = "Hello, world!"
IO.puts(msg)

Hello, world!


:ok

In [4]:
# Rebinding
msg = "foo"
msg = "Hello, world!"

"Hello, world!"

In [5]:
IEx.Info.info(msg)

[{"Data type", "BitString"}, {"Byte size", 13}, {"Description", "This is a string: a UTF-8 encoded binary. It's printed surrounded by\n\"double quotes\" because all UTF-8 encoded code points in it are printable.\n"}, {"Raw representation", "<<72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 33>>"}, {"Reference modules", "String, :binary"}]

In [5]:
^msg = "foo"  # Pinned match without rebinding. This forces a MatchError because msg is already "Hello, world!"

error: no match of right hand side value: "foo"

In [6]:
# And to prove it...
^msg = "Hello, world!"

"Hello, world!"

In [6]:
# Define a function
def hello do
    IO.puts "Hello, world!"
end
hello()

error: cannot invoke def/2 outside module

In [7]:
# Define the function in a module
defmodule Demo do
    def hello() do
        IO.puts "Hello, world!"
    end
end
Demo.hello()

Hello, world!


:ok

In [8]:
defmodule Demo do
    def hello, do: IO.puts "Hello, world!" # Note 1-line syntax
        
    @spec hello_str() :: String.t
    def hello_str, do: "Hello, world!"
end
Demo.hello_str()

"Hello, world!"

In [9]:
defmodule Demo1 do
    def(hello, [{:do, "Hi!"}])
end

{:module, Demo1, <<70, 79, 82, 49, 0, 0, 4, 212, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 137, 0, 0, 0, 14, 12, 69, 108, 105, 120, 105, 114, 46, 68, 101, 109, 111, 49, 8, 95, 95, 105, 110, 102, 111, 95, 95, 10, ...>>, {:hello, 0}}

In [10]:
Demo1.hello()

"Hi!"

In [11]:
# "Lambda"
hello_fn = fn -> IO.puts("Hello, world!") end  # Define it
hello_fn.()  # Call it

Hello, world!


:ok

In [12]:
# Function overloading, sort of
defmodule Demo do
    def hello(), do: IO.puts("Hello, world!")       # This is hello/0
    def hello(name), do: IO.puts("Hello, #{name}!") # This is hello/1
end
Demo.hello("old buddy")

Hello, old buddy!


:ok

In [13]:
# Processes
defmodule Procs do
    def spawn_hello() do
        spawn(
            fn ->
                my_pid = self()
                IO.puts("[#{inspect my_pid}] Hello, world!")
            end
        )
    end
end
Procs.spawn_hello()

#PID<0.193.0>

[#PID<0.193.0>] Hello, world!


In [14]:
self()

#PID<0.171.0>

In [15]:
# Send myself a message from a process
me = self()
spawn(fn -> send(me, {:msg, :hi, {:from, self()}}) end)
receive do
    {:msg, data, source} -> IO.puts("Got msg #{inspect data}, source: #{inspect source}")
    after 1_000 -> :timeout
end

Got msg :hi, source: {:from, #PID<0.194.0>}


:ok

In [16]:
# Time out
receive do
    {:msg, data, source} -> IO.puts("Got msg #{inspect data}, source: #{inspect source}")
    after 1_000 -> :timeout
end

:timeout

In [17]:
# Pattern match a list
[h|t] = [1, 2, 3, 4, 5]
IO.puts("h: #{h}, t: #{inspect t}")
[h|t] = [1]
IO.puts("h: #{h}, t: #{inspect t}")

h: 1, t: [2, 3, 4, 5]
h: 1, t: []


:ok

In [17]:
# What happens if...?
[h|t] = []

error: no match of right hand side value: []

In [18]:
# Append two lists
[1, 2] ++ [3, 4, 5]

[1, 2, 3, 4, 5]

In [19]:
# Push a value onto the head of a list
new_head = 1
list = [2, 3, 4]
new_list = [new_head | list]

[1, 2, 3, 4]

In [20]:
# Alternative way to split up a list
l = [1, 2, 3, 4]
h = hd(l)
t = tl(l)
IO.puts("Head: #{h}, tail: #{inspect t}")

Head: 1, tail: [2, 3, 4]


:ok

### Recursion

In [21]:
# Reverse a list
defmodule MyList do
    # Non-tail-recursive version
    def reverse([h|t]), do: reverse(t) ++ [h]
    def reverse([]), do: []
        
    # Tail-recursive version
    def reverse_t([h|t], acc \\ []), do: reverse_t(t, [h|acc])
    def reverse_t([], acc), do: acc
end

list = [1, 2, 3, 4, 5]
list_rev = MyList.reverse(list)
list_rev_t = MyList.reverse_t(list)
^list_rev = list_rev_t # Force pattern-match

[5, 4, 3, 2, 1]

## Higher-order Functions

In [22]:
Enum.map(1..5, fn x -> x * 2 end)

[2, 4, 6, 8, 10]

In [23]:
Enum.map(1..5, &(&1 * 2)) # using function capture

[2, 4, 6, 8, 10]

In [24]:
Enum.with_index(1..5)

[{1, 0}, {2, 1}, {3, 2}, {4, 3}, {5, 4}]

In [25]:
# Introduce the pipe operator
1..5 |> Enum.with_index

[{1, 0}, {2, 1}, {3, 2}, {4, 3}, {5, 4}]

In [26]:
"the quick brown fox"
|> String.split
|> IO.inspect
|> Enum.map(&String.capitalize/1)
|> IO.inspect
|> Enum.join("")

["the", "quick", "brown", "fox"]
["The", "Quick", "Brown", "Fox"]


"TheQuickBrownFox"

In [27]:
# What it would look like without using the pipe operator
Enum.join(Enum.map(String.split("the quick brown fox"), &String.capitalize/1), " ")

"The Quick Brown Fox"

In [28]:
# Sum of products of 2-tuples
defmodule Demo do
    def sum_prod_tuples(tuples) do
        tuples
        |> Enum.reduce(0, fn {x, y}, acc -> acc + x * y end)
    end
end
101..105 |> Enum.with_index |> Demo.sum_prod_tuples

1040

In [29]:
101..105 |> Enum.with_index |> Enum.each(fn {n, ndx} -> IO.puts("#{ndx}: #{n}") end)

0: 101
1: 102
2: 103
3: 104
4: 105


:ok

In [30]:
Integer.pow(2, 3)

8

In [31]:
Integer.__info__(:functions)

[digits: 1, digits: 2, extended_gcd: 2, floor_div: 2, gcd: 2, mod: 2, parse: 1, parse: 2, pow: 2, to_char_list: 1, to_char_list: 2, to_charlist: 1, to_charlist: 2, to_string: 1, to_string: 2, undigits: 1, undigits: 2]

## Interoperability with Erlang
Let's call Erlang's `application` module to get a list of all running applications.

In [32]:
:application.which_applications()

[{:ierl, 'ierl', '0.6.0'}, {:ex_unit, 'ex_unit', '1.12.3'}, {:logger, 'logger', '1.12.3'}, {:mix, 'mix', '1.12.3'}, {:iex, 'iex', '1.12.3'}, {:lfe, 'Lisp Flavored Erlang (LFE)', '2.0.1'}, {:getopt, 'Command-line options parser for Erlang', '1.0.2'}, {:jupyter, 'jupyter', '0.5.0'}, {:jsx, 'a streaming, evented json parsing toolkit', '3.1.0'}, {:uuid, 'Native UUID Generation', '2.0.1'}, {:quickrand, 'Quick Random Number Generation', '2.0.1'}, {:crypto, 'CRYPTO', '5.0.6'}, {:gproc, 'Extended process registry for Erlang', '0.9.0'}, {:chumak, 'Erlang implementation of ZeroMQ Transport Protocol (ZMTP)', '1.3.0'}, {:iso8601, 'An ISO 8601 date formating and parsing library for Erlang', '1.3.2'}, {:elixir, 'elixir', '1.12.3'}, {:compiler, 'ERTS  CXC 138 10', '8.1.1'}, {:stdlib, 'ERTS  CXC 138 10', '3.17.2'}, {:kernel, 'ERTS  CXC 138 10', '8.3.2'}]

In [33]:
# Let's just get the descriptions using a list comprehension
for {_, desc, _} <- :application.which_applications(), do: desc

['ierl', 'ex_unit', 'logger', 'mix', 'iex', 'Lisp Flavored Erlang (LFE)', 'Command-line options parser for Erlang', 'jupyter', 'a streaming, evented json parsing toolkit', 'Native UUID Generation', 'Quick Random Number Generation', 'CRYPTO', 'Extended process registry for Erlang', 'Erlang implementation of ZeroMQ Transport Protocol (ZMTP)', 'An ISO 8601 date formating and parsing library for Erlang', 'elixir', 'ERTS  CXC 138 10', 'ERTS  CXC 138 10', 'ERTS  CXC 138 10']

## Dictionaries (Maps)

In [34]:
## In Elixir, dictionaries are called "maps" and map literals use the format '%{}' 
map1 = %{"count" => 10, "sum" => 55}

%{"count" => 10, "sum" => 55}

In [35]:
## Map keys can be atoms
map2 = %{:count => 10, :sum => 55}

%{count: 10, sum: 55}

In [36]:
## If map keys are all atoms, the syntax becomes simpler
map3 = %{count: 10, sum: 55}

%{count: 10, sum: 55}

In [37]:
## Get a value from a map
map3[:count]  # Or also map3.count, which raises an exception if the key is absent.

10

In [38]:
## Make a new map with the count incremented (remember, data is immutable)
map4 = %{map3 | count: map3.count + 1}

%{count: 11, sum: 55}

In [39]:
## But map3 hasn't changed
map3

%{count: 10, sum: 55}

In [39]:
## You can't assign like this
map3.count = 22

error: nofile:2: cannot invoke remote function map3.count/0 inside a match

In [40]:
Map.keys(map4)

[:count, :sum]

In [41]:
Map.values(%{one: "two", three: "four"})

["two", "four"]

In [42]:
Map.to_list(map4)

[count: 11, sum: 55]

In [43]:
[{:count, 11}, {:sum, 55}] = [count: 11, sum: 55]  # Same things

[count: 11, sum: 55]

In [44]:
# Iterate over a map using either a comprehension or Enum
Enum.each(map4, fn {k, v} -> IO.puts("k: #{k}, v: #{v}") end)

k: count, v: 11
k: sum, v: 55


:ok

In [45]:
# Using a comprehension
for {k, v} <- map4, do: IO.puts("k: #{k}, v: #{v}")

k: count, v: 11
k: sum, v: 55


[:ok, :ok]

## Keyword lists

In [46]:
defmodule KW do
    def combine(a_list, opts \\ []) when is_list(a_list) do
        case opts[:op] do
            :sum ->
                {:ok, Enum.sum(a_list)}
            :prod ->
                {:ok, List.foldl(a_list, 1, &(&1 * &2))}
            op when op in [:max, nil] ->
                {:ok, Enum.max(a_list)}
            op ->
                {:error, {:invalid_opt, op}}
        end
    end
end

{:module, KW, <<70, 79, 82, 49, 0, 0, 7, 240, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 225, 0, 0, 0, 27, 9, 69, 108, 105, 120, 105, 114, 46, 75, 87, 8, 95, 95, 105, 110, 102, 111, 95, 95, 10, 97, 116, 116, ...>>, {:combine, 2}}

In [47]:
KW.combine(Enum.to_list(1..10))

{:ok, 10}

In [48]:
KW.combine(Enum.to_list(1..10), op: :sum)

{:ok, 55}

In [49]:
KW.combine(Enum.to_list(1..10), op: :prod)

{:ok, 3628800}

In [50]:
KW.combine(Enum.to_list(1..10), op: :foo)

{:error, {:invalid_opt, :foo}}

In [50]:
{:ok, result} = KW.combine(Enum.to_list(1..10), op: :foo)

error: no match of right hand side value: {:error, {:invalid_opt, :foo}}

## GenServer

In [51]:
defmodule Stack do
    use GenServer
    # API
    def push(pid, x), do: GenServer.cast(pid, {:push, x})
    def pop(pid), do: GenServer.call(pid, :pop)
    def empty?(pid), do: GenServer.call(pid, :is_empty)

    @impl true
    def init(stack), do: {:ok, stack}
    @impl true
    def handle_call(:pop, _from, [top|new_state]), do: {:reply, {:ok, top}, new_state}
    def handle_call(:pop, _from, []=state), do: {:reply, {:error, :stack_empty}, state}
    def handle_call(:is_empty, _from, state), do: {:reply, {:ok, state == []}, state}
    @impl true
    def handle_cast({:push, value}, state), do: {:noreply, [value|state]}
end

{:module, Stack, <<70, 79, 82, 49, 0, 0, 16, 152, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 131, 0, 0, 0, 43, 12, 69, 108, 105, 120, 105, 114, 46, 83, 116, 97, 99, 107, 8, 95, 95, 105, 110, 102, 111, 95, 95, 10, ...>>, {:handle_cast, 2}}

In [52]:
{:ok, pid} = GenServer.start_link(Stack, [])

{:ok, #PID<0.207.0>}

In [53]:
Stack.push(pid, 10)

:ok

In [54]:
Stack.push(pid, :hello)

:ok

In [55]:
Process.alive?(pid)

true

In [56]:
Stack.pop(pid)

{:ok, :hello}

In [57]:
Stack.pop(pid)

{:ok, 10}

In [58]:
Stack.pop(pid)

{:error, :stack_empty}

In [59]:
GenServer.stop(pid)

:ok

In [60]:
Process.alive?(pid)

false

## Links
* [Why Elixir & Phoenix is a great choice for your web app in 2022](https://curiosum.com/blog/why-elixir-phoenix-great-choice-for-modern-web-app)
* [5 million concurrent users in Elixir](https://blog.discord.com/scaling-elixir-f9b8e1e7c29b)
* [How We Replaced React with Phoenix](https://thoughtbot.com/blog/how-we-replaced-react-with-phoenix)

## Appendix: A million concurrent processes? Really?
It depends on the number of processes the Erlang kernel is configured to run. By default it is set to a maximum of 262144 processes, but it can be changed at startup to well in excess of 100 million (which I doubt has ever been reached, but you never know). So this Jupyter Notebook erlang kernel will only manage something in the neighborhood of 250K processes because I haven't figured out how to get to the startup parameters.

In [61]:
defmodule HighConcurrency do
    def run(num_procs) do
        parent = self()
        {elapsed, pids} = :timer.tc(fn -> spawn_procs(num_procs, parent) end)
        IO.puts("Started #{num_procs} processes in #{elapsed / num_procs} micros each")
        {elapsed, :ok} = :timer.tc(fn -> stop_procs(pids) end)
        len = length(pids)
        IO.puts("Sent #{len} stop messages in #{elapsed / len} micros each")
        {elapsed, count} = :timer.tc(&receive_loop/0)
        IO.puts("Received #{count} shutdown messages in #{elapsed / count} micros each")
        
        count
    end

    def spawn_procs(num_procs, parent) do
        for proc_num <- 1..num_procs do
            spawn(fn -> wait_for_stop(parent, proc_num) end)
        end
    end

    def wait_for_stop(parent, proc_num) do
        receive do
            :stop -> send(parent, {:stopped, proc_num})
            after 5_000 -> {:error, :timeout}
        end
    end
        
    def stop_procs(pids) do
        Enum.each(pids, fn pid -> send(pid, :stop) end)
    end

    def receive_loop(acc \\ 0) do
        receive do
            {:stopped, _proc_num} ->
                receive_loop(acc + 1)
            _ ->
                receive_loop(acc)
        after 10 ->
            acc
        end
    end

end

IO.puts("Number of processes: #{inspect HighConcurrency.run(250000)}")

Started 250000 processes in 2.798028 micros each
Sent 250000 stop messages in 1.532424 micros each
Received 250000 shutdown messages in 0.515564 micros each
Number of processes: 250000


:ok

### Running this in IEx with max processes set to 2 million

~~~
IO.puts("Number of processes: #{inspect HighConcurrency.run(1000000)}")
Started 1000000 processes in 2.145148 micros each
Sent 1000000 stop messages in 1.477605 micros each
Received 1000000 shutdown messages in 0.462585 micros each
Number of processes: 1000000
~~~